In this notebook we will write a generalized version of Greedy Best First Search methods that can work with any problem once it is formulated. Then, the key of the solution will be generating proper problem formulations and the proper heuristic function.


## Frontier operation Functions
In GBFS methods the extraction is not based on their arrival to the queue order, but on the similarity to the heuristic function. 



## PROBLEM FORMULATION
We will have a couple of evaluation functions based on different heuristics for the P8 problem and check how them affect to the solution


In [1]:
import numpy as np
class ProblemP8 (): 
    #attributes of the class are empty
    name = ""
    initial_state = {}
    goal_state = {}
    actions = []
    rows = 3
    columns= 3
    
    def __init__(self): ## init method is the constructor. 

        # normally we would go with a random initialization
        #numbers =np.array([1,2,5,3,4,8,0,6,7])##--->> this configuration only requires depth 6 solution

        #numbers =np.array([1,2,5,3,6,7,0,4,8]) #---<>> depth 11 solution
        numbers =np.array([1,2,5,4,6,7,3,8,0]) ## ->>>> higher depth

        self.name ="8PuzzleProblem"
        self.initial_state = numbers.reshape(self.rows, self.columns)
        self.goal_state = np.arange(self.rows*self.columns).reshape(self.rows, self.columns)
        self.actions = ["Up", "Down", "Left", "Right"]


    #we need this in case we want other size
    def set_row_cols (self, rows, cols):
        self.rows=rows
        self.columns=cols
    
    def is_final_state(self, state):
        return np.array_equal(state, self.goal_state)

    #####
    def is_applicable (self, state, action):
        #find where is 0 in the state to know if the change is applicable:
        row = np.where(state==0)[0][0] +1
        col = np.where(state==0)[1][0] +1
        if action == "Up":
          return row!=1
        if action == "Down":
          return row!=self.rows
        if action == "Left":
          return col!=1
        if action == "Right":
          return col!=self.columns
   

    # this function should exist for all problems, but its internal operation should be changed per problem
    # in this case, the applicability is checked in the expansion function, so the effect of a change is always produced
    def effect (self, state, action):
        row = np.where(state==0)[0][0]
        col = np.where(state==0)[1][0]
        result = state.copy()  
        if action == "Up":
            result[row-1,col]=state[row,col]
            result[row,col]=state[row-1,col]

        if action == "Down":
          result[row+1,col]=state[row,col]
          result[row,col]=state[row+1,col]
        
        if action == "Left":
          result[row,col-1]=state[row,col]
          result[row,col]=state[row,col-1]
        
        if action == "Right":
          result[row,col+1]=state[row,col]
          result[row,col]=state[row,col+1]
        
        return result
      

  ## COST WILL BE 1 FOR EACH MOVED TILE
    def get_cost(self, action, state):
        return 1
  ### but the evaluation should be some function that tells us how different is the solution

    def get_evaluation2 (self, state): #H1
      # returns the sum of elements that are different to the goal state: we want to bring this evaluation to 0. 
      #return sum(sum(state!=self.goal_state))
      misplaced = 0
      for r in range(self.rows):
          for c in range(self.columns):
            if state[r,c]!=0: #we don't consider the state of 0
              if state[r,c]!= self.goal_state[r,c]:
                misplaced+=1
              
      return misplaced

    

    def manhattan_distance(self, number, row, col, goal_state):
      target_row = np.where(goal_state==number)[0][0]
      target_col = np.where(goal_state==number)[1][0]
      vertical_distance = np.abs(row-target_row)
      horizontal_distance = np.abs(col-target_col)
      return horizontal_distance+vertical_distance

    def get_evaluation (self, state): #H2
      # returns the sum of elements that are different to the goal state: we want to bring this evaluation to 0. 
      total_manhattan_distance = 0
      for r in range(self.rows):
          for c in range(self.columns):
            if state[r,c]!=0: #we don't consider the state of 0
              total_manhattan_distance+=self.manhattan_distance(state[r, c],r, c, self.goal_state)
      return total_manhattan_distance


In [2]:
problem = ProblemP8()
print(problem.initial_state)

print (problem.get_evaluation(problem.initial_state))
print (problem.get_evaluation2(problem.initial_state))

[[1 2 5]
 [4 6 7]
 [3 8 0]]
10
8


# INFORMED SEARCH




## A*


In [3]:
def expand_a_star (node, problem):
    new_nodes = []
    possible_actions = problem.actions
    for action in possible_actions:
        if problem.is_applicable(node["state"], action):
            new_state= problem.effect (node["state"], action)
            new_node = {}
            new_node["state"]=new_state
            new_node["parent_node"]=node
            new_node["actions"]=node["actions"] + [action]
            new_node["cost"]=node["cost"] + problem.get_cost(action, node["state"])
            new_node["depth"]=node["depth"]+1
            ##### REWRITE THE EVALUATION
            new_node["evaluation"]=  new_node["cost"] + problem.get_evaluation(new_state)
            new_nodes.append (new_node)
    return new_nodes

In [4]:
def A_star(problem, iteration_limit):
    # result dictionary
    result = {"method":"A*", "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}
    
    # 1. problem definition
    # problem = Problem()
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":problem.get_evaluation(problem.initial_state)}
    frontier = []

     # 2. add node to frontier
    frontier.append(initial_node)
    
    # 3. start exploring and expanding the frontier
    iterations=1
    depth= 1
    while len (frontier)>0: #if we have elements in the frontier...

        #control if we have reached the iteration limit, stop
        if iterations>iteration_limit:
            result["status"] = "Maximum number of iterations reached"
            break

         # 3.1. get first element of frontier and delete it
        node = frontier[0]
        
        
        frontier = frontier[1:]
        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.
        
        # 3.3 if it is not final, expand and add to the frontier
        
        new_nodes = expand_a_star(node, problem)
        for n in new_nodes:
            frontier.append(n)
            if n["depth"]>depth:
              depth=n["depth"]
              print ("Depth reached=", depth)
        # sorting by ascending order this will lead to 
        frontier = sorted(frontier, key=lambda field: field['evaluation']) 

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations
        
        iterations+=1
      #loop keeps running until no more nodes available or final state obtained
    
    result["final_state"] = node
    return(result)

# COMPARISON


In [5]:
problem = ProblemP8()
def print_results (res):
  print ("METHOD: {} \n Final_state: {} \n Final Status: {} \n Maximum frontier size: {}  \n Maximum Depth: {} \n Iterations reached: {}  \n COST:{}".format(res["method"], res["final_state"]["actions"], res["status"], res["max_frontier"], res["max_depth"], res["iterations"],res["final_state"]["cost"]))
[1,2,5,4,6,7,3,8,0]

[1, 2, 5, 4, 6, 7, 3, 8, 0]

##P8 PROBLEM WITH A*
HEURISTIC: MANHATTAN

HEURISTIC: BAD POSITIONED TILES





In [6]:
a_star_= A_star(problem, iteration_limit=200000)
print_results (a_star_ )

Depth reached= 2
Depth reached= 3
Depth reached= 4
Depth reached= 5
Depth reached= 6
Depth reached= 7
Depth reached= 8
Depth reached= 9
Depth reached= 10
Depth reached= 11
Depth reached= 12
Depth reached= 13
Depth reached= 14
Depth reached= 15
Depth reached= 16


KeyboardInterrupt: 

In [ ]:
import numpy as np
goal_state = np.arange(3*3).reshape(3,3)
def get_evaluation1 (state, goal_state): #H1
  # returns the sum of elements that are different to the goal state: we want to bring this evaluation to 0. 
  misplaced = 0
  for r in range(3):
    for c in range(3):
      if state[r,c]!=0: #we don't consider the state of 0
        if state[r,c]!=goal_state[r,c]:
          misplaced+=1
  return misplaced


def manhattan_distance(number, row, col, goal_state):
  target_row = np.where(goal_state==number)[0][0]
  target_col = np.where(goal_state==number)[1][0]
  vertical_distance = np.abs(row-target_row)
  horizontal_distance = np.abs(col-target_col)
  return horizontal_distance+vertical_distance

def get_evaluation2 (state, goal_state): #H2
  # returns the sum of elements that are different to the goal state: we want to bring this evaluation to 0. 
  total_manhattan_distance = 0
  for r in range(3):
      for c in range(3):
        if state[r,c]!=0: #we don't consider the state of 0
          total_manhattan_distance+=manhattan_distance(state[r, c],r, c, goal_state)
  return total_manhattan_distance

def straight_line(number, row, col, goal_state):
  target_row = np.where(goal_state==number)[0][0]
  target_col = np.where(goal_state==number)[1][0]
  vertical_distance = np.abs(row-target_row)
  horizontal_distance = np.abs(col-target_col)
  return np.sqrt(vertical_distance**2+horizontal_distance**2)

def get_evaluation3 (state, goal_state):
  distance = 0
  for r in range(3):
    for c in range(3):
      if state[r,c]!=0: #we don't consider the state of 0
        distance+=straight_line(state[r, c],r, c, goal_state)
  return distance


numbers =np.array([1,2,5,3,4,8,0,6,7])##--->> this configuration only requires depth 6 solution

#numbers =np.array([1,2,5,3,6,7,0,4,8]) #---<>> depth 11 solution
#numbers =np.array([1,2,5,4,6,7,3,8,0]) ## ->>>> higher depth
misps = []
manhs = []
stras = []
for i in range(100):
  np.random.shuffle(numbers)
  state = numbers.reshape(3, 3)
  print (state)
  print ("misplaced ", get_evaluation1(state, goal_state))
  print ("manhattan ", get_evaluation2(state, goal_state))
  print ("straight ", get_evaluation3(state, goal_state))
  misps.append(get_evaluation1(state, goal_state))
  manhs.append(get_evaluation2(state, goal_state))
  stras.append(get_evaluation3(state, goal_state))

print ("AVERAGE")
print ("misplaced ", np.mean(misps))
print ("manhattan ", np.mean(manhs))
print ("straight ",  np.mean(stras))



[[0 7 6]
 [4 5 8]
 [1 2 3]]
misplaced  8
manhattan  18
straight  14.53663105724556
[[5 2 3]
 [1 4 7]
 [8 0 6]]
misplaced  7
manhattan  15
straight  12.30056307974577
[[0 5 8]
 [7 6 1]
 [3 2 4]]
misplaced  8
manhattan  16
straight  12.307135789365265
[[7 2 0]
 [4 5 1]
 [3 6 8]]
misplaced  7
manhattan  10
straight  8.650281539872886
[[8 1 6]
 [2 4 5]
 [3 7 0]]
misplaced  4
manhattan  12
straight  8.89292222699217
[[0 4 6]
 [1 3 5]
 [7 8 2]]
misplaced  7
manhattan  12
straight  10.242640687119286
[[1 8 5]
 [3 2 7]
 [4 6 0]]
misplaced  7
manhattan  12
straight  9.478708664619075
[[3 6 0]
 [8 1 4]
 [5 2 7]]
misplaced  8
manhattan  16
straight  12.94427190999916
[[7 4 3]
 [6 0 1]
 [5 8 2]]
misplaced  8
manhattan  16
straight  13.122417494872465
[[2 6 5]
 [3 7 4]
 [8 0 1]]
misplaced  7
manhattan  13
straight  11.47213595499958
[[2 8 3]
 [4 0 6]
 [7 5 1]]
misplaced  8
manhattan  18
straight  14.358485472372255
[[3 0 8]
 [5 4 1]
 [2 7 6]]
misplaced  6
manhattan  13
straight  11.242640687119286
